In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib as plt


nltk.download('stopwords')
stopwords = stopwords.words('english')

In [2]:
true_df = pd.read_csv('../datasets/fake-news/True.csv')
fake_df = pd.read_csv('../datasets/fake-news/Fake.csv')

true_df = true_df.assign(fake = [False for _ in true_df.iterrows()])
fake_df = fake_df.assign(fake = [True for _ in fake_df.iterrows()])

df = pd.concat([true_df, fake_df])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 0 to 23480
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   fake     44898 non-null  bool  
dtypes: bool(1), object(4)
memory usage: 1.8+ MB


In [3]:
df = df.reset_index(drop=True)

def clean_text(text: str) -> str:
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
    BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
    STOPWORDS = set(stopwords)
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

In [4]:
df['text'] = df['text'].apply(clean_text)
df['text'] = df['text'].str.replace('\d+', '')

C:\Users\krick\anaconda3\envs\nm\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [18]:
df.columns

Index(['title', 'text', 'subject', 'date', 'fake'], dtype='object')

# GloVe

In [5]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

In [7]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(df.text)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(df.text)
print(encoded_docs[:10])

[[32, 11, 316, 199, 18, 5930, 3, 100, 647, 164, 1058, 2760, 44, 846, 336, 94, 1090, 82, 1199, 199, 228, 942, 375, 7693, 1582, 3127, 7549, 76, 204, 56, 3, 653, 772, 7302, 535, 1780, 369, 250, 1538, 437, 467, 81, 574, 303, 16440, 1240, 284, 623, 7659, 103, 303, 1264, 244, 783, 81, 375, 387, 196, 1792, 308, 215, 115, 35, 380, 442, 29, 1591, 1311, 56, 992, 270, 245, 100, 5, 12, 2, 56, 91, 289, 375, 806, 83, 574, 110, 9, 91, 11696, 3301, 16213, 12363, 574, 840, 60, 851, 3859, 922, 1289, 78, 203, 1209, 1040, 2, 61, 188, 1166, 45, 54, 806, 16213, 12363, 574, 53, 7302, 465, 627, 3232, 14, 641, 1978, 1, 217, 110, 90, 1491, 337, 129, 273, 15, 336, 1146, 53, 1199, 199, 843, 93, 10693, 1671, 381, 4858, 167, 1, 7302, 204, 56, 647, 493, 481, 1715, 34130, 94, 2169, 319, 10860, 81, 375, 2106, 1810, 2130, 37, 2130, 44, 846, 2145, 998, 772, 473, 1199, 1282, 59, 3, 653, 2610, 11530, 1, 1780, 11530, 1, 18, 94, 62, 4, 1446, 20, 13, 9424, 2130, 713, 415, 4437, 1167, 94, 1090, 2196, 1176, 7, 232, 13783, 996,

In [8]:
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[   67   251   647   795]
 [ 7332 16667  1795  1972]
 [ 1384     3  2706     1]
 ...
 [  618   476   181  2363]
 [  618   476  6234   371]
 [  618   476   169  2363]]


In [12]:
import zipfile
import urllib.request

pth = 'glove.6B.zip'
output_txt = 'glove.6B.txt'
urllib.request.urlretrieve('https://nlp.stanford.edu/data/glove.6B.zip',pth)
with zipfile.ZipFile(f'./{pth}', 'r') as zip_ref:
    zip_ref.extractall('glove.6B.txt')

In [14]:
# load the whole embedding into memory
embeddings_index = dict()

f = open('./glove.6B.txt/glove.6b.100d.txt', encoding="utf8")
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [15]:
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [16]:
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            20904100  
                                                                 
 flatten (Flatten)           (None, 400)               0         
                                                                 
 dense (Dense)               (None, 1)                 401       
                                                                 
Total params: 20,904,501
Trainable params: 401
Non-trainable params: 20,904,100
_________________________________________________________________
None


In [19]:
# fit the model
model.fit(padded_docs, df.fake, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, df.fake, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 88.832462
